In [35]:
import requests
import json
from datetime import datetime
from typing import Optional, List, Dict, Any, Generator


from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

BASE_URL = "http://localhost:8765"
headers = {"Authorization": f"Bearer {vexa.token}", "Content-Type": "application/json"}

def submit_token(token: str) -> Dict:
    return requests.post(f"{BASE_URL}/submit_token", json={"token": token}).json()

def chat(query: str, thread_id: Optional[str] = None, model: str = "gpt-4o-mini", temperature: float = 0.7) -> Dict:
    response = requests.post(f"{BASE_URL}/chat", headers=headers, json={
        "query": query, "thread_id": thread_id, "model": model, "temperature": temperature
    }, stream=True)
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}
def global_search(query: str, limit: int = 200, min_score: float = 0.4) -> Dict:
    return requests.post(f"{BASE_URL}/search/global", headers=headers, 
        json={"query": query, "limit": limit, "min_score": min_score}).json()

def search_transcripts(query: str, meeting_ids: Optional[List[str]] = None, min_score: float = 0.8) -> Dict:
    return requests.post(f"{BASE_URL}/search/transcripts", headers=headers,
        json={"query": query, "meeting_ids": meeting_ids, "min_score": min_score}).json()

def get_meetings(offset: int = 0, limit: int = 50) -> Dict:
    return requests.get(f"{BASE_URL}/meetings/all", headers=headers, 
        params={"offset": offset, "limit": limit}).json()

def get_meeting_details(meeting_id: str) -> Dict:
    return requests.get(f"{BASE_URL}/meeting/{meeting_id}/details", headers=headers).json()

def create_share_link(target_email: str, access_level: str = "READ", 
                     expiration_hours: int = 24, include_existing: bool = True) -> Dict:
    return requests.post(f"{BASE_URL}/share-links", headers=headers, json={
        "access_level": access_level, "target_email": target_email,
        "expiration_hours": expiration_hours, "include_existing_meetings": include_existing
    }).json()

def accept_share_link(token: str, accepting_email: str) -> Dict:
    return requests.post(f"{BASE_URL}/share-links/accept", headers=headers,
        json={"token": token, "accepting_email": accepting_email}).json()

def start_indexing(num_meetings: int = 200) -> Dict:
    return requests.post(f"{BASE_URL}/start_indexing", headers=headers,
        json={"num_meetings": num_meetings}).json()

def get_indexing_status() -> Dict:
    return requests.get(f"{BASE_URL}/indexing_status", headers=headers).json()

# Add these functions to the initialization cell with other API functions
def get_threads(meeting_id: Optional[str] = None) -> Dict:
    """Get threads for global or meeting-specific context"""
    if meeting_id:
        return requests.get(f"{BASE_URL}/threads/{meeting_id}", headers=headers).json()
    return requests.get(f"{BASE_URL}/threads", headers=headers).json()

def delete_thread(thread_id: str) -> Dict:
    return requests.delete(f"{BASE_URL}/thread/{thread_id}", headers=headers).json()

def chat_meeting(
    query: str, 
    meeting_ids: List[str], 
    thread_id: Optional[str] = None, 
    model: str = "gpt-4o-mini", 
    temperature: float = 0.7
) -> Dict:
    response = requests.post(
        f"{BASE_URL}/chat/meeting", 
        headers=headers, 
        json={
            "query": query,
            "meeting_ids": meeting_ids,
            "thread_id": thread_id,
            "model": model,
            "temperature": temperature
        },
        stream=True
    )
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [36]:
r =chat('dmitry')

Dmitriy Grankin has been actively involved in various discussions and projects, focusing on the development of tools and systems aimed at improving efficiency and accessibility in meetings and legal processes. Below are key highlights of his contributions:

### Key Contributions

- **API Development**: Completed the development of an API for integration with frontend systems, enhancing functionality and user interaction[1][4].
  
- **Frontend Development**: Actively working on the frontend for a project that interacts with the API, addressing functionality and usability issues[1][4].

- **Testing LAMA 3.2**: Conducted tests on the LAMA 3.2 model, which has 90 billion parameters, showing promising results but requiring further refinement for project use[1][4].

- **Knowledge Base Creation**: Finalized the delivery of a knowledge base compiling information from Google Meet calls, making it accessible for future reference and use[2][4].

- **Search Capabilities Implementation**: Researche

In [38]:
r = chat_meeting('о чем встреча?', ['ab27bb1f-93d5-4500-9865-6f5de4d433ea'])

Встреча была посвящена переходу на новый dashboard и интеграции с маркетингом. Основные обсуждаемые темы включали:

1. Необходимость наличия нормального локального счета и поиска summary и индекса для успешного перехода на новый dashboard.
2. Отладка системы Threads и создание контекстного summary из глобального контекста.
3. Планы по интеграции с email-маркетингом и календарями, а также выпуск хотя бы одного поста по маркетингу.
4. Обсуждение возможности переноса локального серчборда и необходимость использования более широкого контекста для векторного поиска.

В целом, встреча сосредоточена на улучшении функциональности и интеграции различных систем для повышения эффективности работы.

In [31]:
r

{'thread_id': '269f19f0-ef3f-46aa-9adb-d28d4910bfdd',
 'service_content': {'context_source': 'meeting',
  'meeting_count': 1,
  'meeting_id': 'ab27bb1f-93d5-4500-9865-6f5de4d433ea'}}